In [0]:
# Create widgets for parameters
dbutils.widgets.text('datefrom', '')
dbutils.widgets.text('dateto', '')

datefrom_p = dbutils.widgets.get('datefrom')
dateto_p = dbutils.widgets.get('dateto')

# make them fit to eBay standards
datefrom = datefrom_p + '.000Z'
dateto = dateto_p + '.000Z'

In [0]:
# Get the Access Token by using Refresh Token
import pandas as pd
import requests
import json
from time import sleep
import gzip


refresh_token  = 'put_refresh_token_here'

def get_access_token():
    global refresh_token
    url = "https://api.ebay.com/identity/v1/oauth2/token"
    headers = {
               "Content-Type": "application/x-www-form-urlencoded ; charset=utf-8",
               "Authorization": "Basic  base64coded(Client ID:Client Secret)"
              }
    data = {
            "grant_type":"refresh_token",
            "refresh_token" :refresh_token
           }
 
    response = requests.post(url = url, headers = headers, data = data)
    return response.json()['access_token']

access_token = get_access_token()

In [0]:
# Post request to generate report
g_url = "https://api.ebay.com/sell/marketing/v1/ad_report_task"

g_headers= {
           'Authorization': "Bearer " + access_token,
           'Content-Type': 'application/json'
           }

g_data = {
        "reportType": "CAMPAIGN_PERFORMANCE_REPORT",
        "marketplaceId": "EBAY_DE",
        "dateFrom": datefrom,
        "dateTo": dateto,
        "fundingModels": ["COST_PER_CLICK"],
        "campaignIds": [],                          
        "reportFormat": "TSV_GZIP",
        "metricKeys": [
                    "cpc_impressions",
                    "cpc_clicks",
                    "cpc_ctr",
                    "cpc_attributed_sales",
                    "cpc_conversion_rate",
                    "cpc_sale_amount_listingsite_currency",
                    "cost_per_click",
                    "cpc_avg_cost_per_sale",
                    "cpc_ad_fees_listingsite_currency",
                    "cpc_return_on_ad_spend",
                    "cpc_ad_fees_payout_currency",
                    "cpc_sale_amount_payout_currency"
                     ],
        "dimensions": [
                            {
                                "annotationKeys": [ "campaign_name","campaign_start_date", "campaign_end_date" ],
                                "dimensionKey": "campaign_id"
                            },
                            {
                                "annotationKeys": [ "ad_group_name" ],
                                "dimensionKey": "ad_group_id"
                            },
                            {
                                "annotationKeys": [ "listing_title","listing_quantity_sold", "listing_quantity_available" ],
                                "dimensionKey": "listing_id"
                            },
                            {
                                "dimensionKey": "inventory_reference_id"
                            },
                            {
                                "dimensionKey": "day"
                            }
                      ] }


g_data = json.dumps(g_data)
generate = requests.post(url = g_url, headers = g_headers, data = g_data)
print(generate)

<Response [202]>

In [0]:
# Get the ReportId from the last generated report.
sleep(120)
r_url = "https://api.ebay.com/sell/marketing/v1/ad_report_task"

r_headers= {
           'Authorization': "Bearer " + access_token
           }

reports = requests.get(url = r_url, headers = r_headers)
reportId = reports.json()['reportTasks'][0]['reportId']
print(reportId)

41902510011

In [0]:
# Download the Report
d_url = "https://api.ebay.com/sell/marketing/v1/ad_report/{}".format(reportId)

d_headers= {
           'Authorization': "Bearer " + access_token
           }

report = requests.get(url = d_url, headers = d_headers)
zipfile = report.content
file = (gzip.decompress(zipfile).decode('utf-8'))

In [0]:
# Connect to Blob Storage and Mounting. 
# sas token can be taken from the container settings > shared access tokens.
# Permissions should be given Read, Write, Delete ..(sp=racwdli&st) while generating sas token. Give expire date range widely so that dont get auth error. 
containerName = "ebay-api-de"
storageAccountName = "blobads"
sas = "sp=racwdli&st=20..............................%3D"
config = "fs.azure.sas." + containerName+ "." + storageAccountName + ".blob.core.windows.net"

try:
    dbutils.fs.unmount("/mnt/ebay-data-api")
except:
    pass
    
try:
    dbutils.fs.mount(
      source = "wasbs://{}@{}.blob.core.windows.net".format(containerName,storageAccountName),
      mount_point = "/mnt/ebay-data-api",
      extra_configs = {config : sas})
except:
    pass

/mnt/ebay-data-api has been unmounted.

In [0]:
# %fs mounts

In [0]:
# Put file to the blob container
dbutils.fs.put('dbfs:/mnt/ebay-data-api/' + datefrom_p.replace(":", "_") + '__' + dateto_p.replace(":", "_") + '.tsv', file)

Wrote 18126 bytes.
Out[93]: True